# Eleições para segundo turno em Duque de Caxias - RJ

Antes de qualquer código e gráfico, vamos a alguns conceitos que serão colocados nas análises a seguir:

As eleições para os cargos executivos (prefeito, governador e presidente) são eleições majoritárias, isto é, o candidato eleito será o que tiver mais votos. Este trabalho visa a analisar, graficamente, dados além dos votos válidos, que são os votos brancos e nulos, e abstenções.

Dois fatos chamam a atenção quando analisa-se a eleição referida:
* A proporção de eleitores que se absteram de votar, somados os que anularam seus votos, foi considerável (mais de 30%)
* A soma desses potenciais votos, foi maior que a soma dos votos válidos para o candidato eleito

Assim, os dados a seguir, acabam por explicitar a problemática da eleição no Brasil, a qual desconsidera os votos inválidos e abstenções na hora da apuração dos votos.

In [1]:
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
from eleicoes_analysis import *

df_detalhe = pd.read_csv('dataset/detalhe_votacao_secao_DC_2016_RJ.csv')
df_votacao_candidato = pd.read_csv('dataset/votacao_munzona_DC_2016_RJ.csv')

numeros_consolidados = getConsolidateNumbers(df_detalhe)
total_votos_validos = numeros_consolidados.get('validos')
total_votos_nao_considerados = numeros_consolidados.get('nao_considerados')
total_abstencoes = numeros_consolidados.get('abstencoes')
total_brancos = numeros_consolidados.get('brancos')
total_nulos = numeros_consolidados.get('nulos')

numeros_consolidados_por_candidato = getConsolidateNumbersByCandidate(df_votacao_candidato)
nameVotes = getNameVotesByCandidate(numeros_consolidados_por_candidato)
cand1 = nameVotes.get('cand1')
cand2 = nameVotes.get('cand2')

## Visualizando os dados

### Visualizando os dados em geral

#### Proporção de eleitores considerados (votos válidos) em relação aos não considerados

In [2]:
values = [total_votos_validos, total_votos_nao_considerados]
labels = [ 'Computados', 'Desconsiderados' ]

title = 'O eleitorado divido em: votos computados e desconsiderados da apuração'
fig = generatePieChart(go=go, labels=labels, values=values, title=title)
py.iplot(fig)

#### Visualização da distribuição dos votos dos candidatos em segundo turno

In [3]:
values = [total_votos_nao_considerados, cand1.get('votes'), cand2.get('votes')]
labels = [ 'Não Computados', cand1.get('name'), cand2.get('name') ]

title = 'O eleitorado divido em: votos por candidato e desconsiderado na apuração'
fig = generatePieChart(go=go, labels=labels, values=values, title=title)
py.iplot(fig)

#### Como a eleição considera votos válidos, como ficou o resultado?

In [4]:
values = [cand1.get('votes'), cand2.get('votes')]
labels = [cand1.get('name'), cand2.get('name')]

title = 'A divisão dos votos válidos na eleição em 2016'
fig = generatePieChart(go=go, labels=labels, values=values, title=title)
py.iplot(fig)

#### Visualizando a divisão entre os não considerados

In [5]:
values = [total_votos_validos, total_abstencoes, total_brancos, total_nulos]
labels = [ 'Válidos', 'Abstenções', 'Brancos', 'Nulos' ]

title = 'Proporcao de eleitores em votos válidos e desconsiderados na apuração'
fig = generatePieChart(go=go, labels=labels, values=values, title=title)
py.iplot(fig)

#### Visualização com a distribuição dos votos dos candidatos em segundo turno

In [6]:
values = [cand1.get('votes'), cand2.get('votes'), total_abstencoes, total_brancos, total_nulos]
labels = [ cand1.get('name'), cand2.get('name'), 'Abstenções', 'Brancos', 'Nulos' ]

title = 'Proporcao de eleitores em votos por candidato e desconsiderados na apuração'
fig = generatePieChart(go=go, labels=labels, values=values, title=title)
py.iplot(fig)

### Visualizando os dados por zona

#### Proporção de eleitores considerados (votos válidos) em relação aos não considerados

In [7]:
agrupado_por_zona = getConsolidateNumbersByZona(df_detalhe)
zonas = agrupado_por_zona.index.values

_zonas = list(map(lambda z : 'Z-{}'.format(z), zonas))

props = ['votos_nominais', 'nao_considerados']
labels = { 'votos_nominais': 'Válidos', 'nao_considerados': 'Não Considerados' }
title = 'Proporção de eleitores em votos válidos e não considerados'
traces = generateDataToStackedBar(go=go, x=_zonas, props=props, labels=labels, df=agrupado_por_zona)
fig = generatedStackedBar(go=go, data=traces, title=title, x='Zonas', y='% de registros')
py.iplot(fig)

#### Visualizando a divisão entre os não considerados

In [8]:
title = 'Proporção de eleitores em votos válidos e divisão dos não considerados'
props = ['votos_nominais', 'abstencoes', 'votos_brancos', 'votos_nulos']
labels = { 'votos_nominais': 'Nominais', 'abstencoes' : 'Abstencoes', 'votos_brancos': 'Brancos', 'votos_nulos': 'Nulos' }
traces = generateDataToStackedBar(go=go, x=_zonas, props=props, labels=labels, df=agrupado_por_zona)
fig = generatedStackedBar(go=go, data=traces, title=title, x='Zonas', y='% de registros')
py.iplot(fig)

#### Visualização com a distribuição dos votos dos candidatos em segundo turno em relação aos votos não considerados

In [9]:
dfl = getConsolidateInformationByZona(agrupado_por_zona, df_votacao_candidato).groupby('zona').sum()

title = 'Proporção de eleitores que votaram em algum candidato e os que não foram considerados'
props = ['nao_considerados', 'DICA', 'WASHINGTON REIS']
labels = {
    'nao_considerados' : 'Não considerados',
    'DICA': 'DICA',
    'WASHINGTON REIS': 'WASHINGTON REIS'
}
traces = generateDataToStackedBar(go=go, x=_zonas, props=props, labels=labels, df=dfl)
fig = generatedStackedBar(go=go, data=traces, title=title, x='Zonas', y='% de registros')
py.iplot(fig)

#### Visualização com a distribuição dos votos dos candidatos em segundo turno em relação aos demais votos

In [10]:
title = 'Proporção de eleitores que votaram em algum candidato em votos brancos, nulos e abstenções'
props = ['abstencoes', 'votos_brancos', 'votos_nulos', 'DICA', 'WASHINGTON REIS']
labels = {
    'abstencoes' : 'Abstencoes',
    'votos_brancos': 'Brancos',
    'votos_nulos': 'Nulos',
    'DICA': 'DICA',
    'WASHINGTON REIS': 'WASHINGTON REIS'
}
traces = generateDataToStackedBar(go=go, x=_zonas, props=props, labels=labels, df=dfl)
fig = generatedStackedBar(go=go, data=traces, title=title, x='Zonas', y='% de registros')
py.iplot(fig)

### Visualizando os dados por seção

O objetivo aqui é trazer as dez seções com mais eleitores aptos. Infelizmente, na data de desenvolvimento desse notebook (Novembro de 2018), não se tinha um dataset com a informação de votação por candidato por seção. Portanto, tem-se as visualizações comparando os votos válidos com os não considerados.

#### Proporção de eleitores considerados (votos válidos) em relação aos não considerados

In [11]:
# ordenando por quantidade de aptos
sorted = df_detalhe.sort_values(by='aptos', ascending=False)
items_grouped = getConsolidateNumbersByZonaSecao(sorted.iloc[:10])
zonas_secao = list(map(lambda x : '(z, s) - {}'.format(x), items_grouped.index.values))
props = ['votos_nominais', 'nao_considerados']
labels = { 'votos_nominais': 'Válidos', 'nao_considerados': 'Não Considerados' }
title = 'Proporção de eleitores em votos válidos e não considerados na apuração (por seção)'
traces = generateDataToStackedBar(go=go, x=zonas_secao, props=props, labels=labels, df=items_grouped)
fig = generatedStackedBar(go=go, data=traces, title=title, x='Zonas', y='% de registros')
py.iplot(fig)

#### Visualizando a divisão entre os não considerados

In [12]:
title = 'Proporção de eleitores em votos válidos e não considerados na apuração (por seção)'
props = ['votos_nominais', 'abstencoes', 'votos_brancos', 'votos_nulos']
labels = { 'votos_nominais': 'Nominais', 'abstencoes' : 'Abstencoes', 'votos_brancos': 'Brancos', 'votos_nulos': 'Nulos' }
traces = generateDataToStackedBar(go=go, x=zonas_secao, props=props, labels=labels, df=agrupado_por_zona)
fig = generatedStackedBar(go=go, data=traces, title=title, x='Zonas', y='% de registros')
py.iplot(fig)

## Referências

Resultado das eleições em [dados.gov](http://dados.gov.br/dataset/resultado-das-eleicoes)